<h1 style="text-align: center;  font-weigth: bold; margin-top: 20px;">
  Visual exploratory data analysis with python and pydeck
</h1>
<h2 style="text-align: center; margin-bottom: 10px;">
  Lorenzo Perozzi
</h2>
<p style="text-align: center; margin-bottom: 20px;">
   <a href="https://www.unige.ch/ge-rgba/welcome/">GE-RGBA Group, Departement of Earth Science, University of Geneva</a>
</p>
<div align="center" style="margin-bottom: 20px;">
    <img src="src/logo.png" style="height: 70px">
</div>


 
## About

During this tutorial, it will be shown how to use python to process and visualize a dataset about geothermal probes availability in the Canton of Geneva. This dataset can be obtained (open access) through the <a href="https://ge.ch/sitg/fiche/6867 ">SITG</a> (Système d’Information du Territoire à Genève). We will cover these a spects:

- Load the geothermal probes dataset.
- Convert CRS coordinate system with **pyproj**
- Extract depth information form attributes
- Visualising the results with **pydeck**

|         | Info |
|--------:|:-----|
| When | Friday, October 1st • 12:30 - 13:00  |
| Live stream | Registration mandatory - [Under this link](https://formulaire.unige.ch/outils/limesurvey3/index.php/268393?lang=fr)  |
| ~~conda environment  | `DST-geothermal-visual`~~ |
| Deck.gl documentation | https://deck.gl/ |


## Setup

If you want to follow along the tutorial, make sure you've done these steps **before the tutorial begin**:

### Step 1

**Install a Python distribution:**

In this tutorial we will be using the [Anaconda](https://www.anaconda.com/)
Python distribution along with the `conda` package manager. If you already have
Anaconda or Miniconda installed, you can skip this step.

If not, please follow the instructions for getting Anaconda up and running in
your system: https://docs.anaconda.com/anaconda/install/

### Step 2

**Create the `DST-geothermal-visual` conda environment:**

1. Clone this repository
1. Open a terminal (*Anaconda Prompt* if you are running Windows). The
   following steps should be done in the terminal.
1. Navigate to the folder that has been cloned
   (if you don't know how to do this, take a moment to read [the Software
   Carpentry lesson on the Unix shell](http://swcarpentry.github.io/shell-novice/)).
1. ~~Create the conda environment by running `conda env create -f environment.yml`
   (this will download and install all of the packages used in the tutorial).~~  
1. **Windows users:** Make sure you set a default browser that is **not Internet Explorer**.
1. ~~Activate the conda environment: `conda activate DST-geothermal-visual`~~
1. ~~Create a new kernel for this environment environment: `ipython kernel install --user --name=DST-geothermal-visual`~~
1. Start the JupyterLab server: `jupyter lab`

1. Start a new terminal from `jupyter lab`  
![](src/terminal1.png)
![](src/terminal2.png)
1. Type the following command inside the terminal: `pip install pandas numpy matplotlib pydeck pyproj`
1. Open the `Visual analyisis of geothermal probes with pydeck.ipynb` to follow the tutorial or a new fresh Notebook if you want to start form scratch.  
1. Feel free to open an issue if you have some problem during the installation or during the tutorial.
   
   
<div style="text-align: left; margin-bottom: 100px;">

 </div>

Importing packages

In [ ]:
import pydeck
import pandas as pd
import numpy as np
import matplotlib.cm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.dpi'] = 150

## Load and inspect the geothermal probes dataset
The dataset is stored on the [SITG](https://ge.ch/sitg/fiche/6867) (Système d'information du territoire à Genève) and publicly available. 

In [ ]:
filename = 'data/CTSS_CHAUFFAGE_SONDE.csv'

To open the dataset we use the [`Pandas`](https://pandas.pydata.org/docs/) package. By default in a `csv` file, columns are separated by a comma `,`, however, we can specify which method is used to spearate the columns. Since the columns in this files are separated by a semicolon, we need to specify the `sep` arguments in the `pd.read_csv` class.

In [ ]:
sondes = pd.read_csv(filename, encoding='latin-1', sep=';')

We can inspect the dataset. For example we can inspect the first 5 lines of the file

In [ ]:
sondes.head(5)

By using the `.info()` methods we can retrieve more details about the dataset such as the total number of rows and columns, the type of the columns and so on.

In [ ]:
sondes.info()

The file contains 11 columns (attributes) and 2033 entries. The objective here is to analyse/visualize the geothermal probes using `pydeck`, an high-scale spatial rendering powered by [deck.gl](https://deck.gl/#/). We need at least 3 attributes: the spatial coordinates (`X` and `Y`) as well as the depth of each probes.

In the dataset the only attributes that do not have `NaN` values are `E` and `N` corresponding to the Easting and Northing coordinates.

In [ ]:
sondes[['E','N']].head(5)
# print(tabulate(sondes[['E','N']].head(5), headers='keys', tablefmt='rst'))

`E` and `N` are in the CH1903+_LV95 reference system (also known as EPSG:2056). 
To visualize the data with pydeck (and in general with all geospatial visualization packages) however, we need to work on *World Geodetic System 1984 - WGS84* (also know as EPSG:4326).

The first step will then to reproject the data from the EPSG:2056 system to the EPSG:4326 system.

## Reproject coordinate from EPSG:2056 to EPSG:4326

There are several online converters for that such as [EPSG.io](https://epsg.io/), which works well when we have few set of coordinates to transform. However, to be efficient, we use `pyproj` that is a Python interface to [PROJ](https://proj.org/) (cartographic projections and coordinate transformation library).

In [ ]:
from pyproj import Transformer
from pyproj import CRS

1. Initializing CRS and create a transformer to convert from 2056 to 4326

In [ ]:
crs_4326 = CRS.from_epsg(4326)
crs_2056 = CRS.from_epsg(2056)
transformer = Transformer.from_crs(crs_2056, crs_4326)

2. Convert `E` and `N` to `Latitude` and `Longitude` respectively

In [ ]:
sondes['lat'], sondes['lon'] = transformer.transform(sondes.E.values, sondes.N.values)

In [ ]:
sondes[['E','N','lat','lon']].head(5)
# print(tabulate(sondes[['E','N','lat','lon']].head(5), headers='keys', tablefmt='rst'))

## Retrieve depth of geothermal probes

We want also know the depth of the geothermal probes. In the dataset we do not have an attribute for that, we need further inspection. Among the 11 columns we have `ALTITUDE_MIN`, `ALTITUDE_MAX` and `DIMENSION` that should give us this information. However these attributes contains several `NaN`. Let's inspect these attributes. 


In [ ]:
sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION']].sample(8, random_state=51)
# print(tabulate(sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION']].sample(8, random_state=51), headers='keys', tablefmt='rst'))

We have several cases: 
1. `NaN` everywhere, we will drop these lines while useless
1. Depth information contained in the `DIMENSION` attributes, as a string text
1. Depth information contained in the (`ALTITUDE_MAX` - `ALTITUDE_MIN`) attributes, as a string text

Note: Normally, with `pd.read_csv()` the `NaNs` are automatically detected, however, depending on how the data have been compiled, it is not always the case. For example, the `0.0` value in `ALTITUDE_MAX` or `ALTITUDE_MIN` attribute, corresponds clearly to `NaNs` values. 

The next step will be to clean up all lines that contains `NaN`.


In [ ]:
sondes.replace(to_replace=0.0, value=np.NaN, inplace=True)
sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION']].sample(8, random_state=51)
# print(tabulate(sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION']].sample(8, random_state=51), headers='keys', tablefmt='rst'))

Then we remove all rows with `NaN` in `ALTITUDE_MAX`, `ALTITUDE_MIN`, `DIMENSION`

In [ ]:
sondes.dropna(subset=['ALTITUDE_MIN', 'ALTITUDE_MAX', 'DIMENSION'], thresh=2, inplace=True)
sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION']].sample(8, random_state=51)
# print(tabulate(sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION']].sample(8, random_state=51), headers='keys', tablefmt='rst'))

We then need to create a `DEPTH` attributes that retrieve geothermal probe information from `ALTITUDE_MIN` and `ALTITUDE_MAX`, if these attributes do not contain `NaN`...

In [ ]:
sondes.reset_index(inplace=True)
idx1 = np.where((~np.isnan(sondes['ALTITUDE_MIN'].values)) & (~np.isnan(sondes['ALTITUDE_MAX'].values)))
sondes['DEPTH'] = np.nan
sondes['DEPTH'].loc[idx1] = sondes.ALTITUDE_MAX - sondes.ALTITUDE_MIN
sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION', 'DEPTH']].sample(8, random_state=2)
# print(tabulate(sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION','DEPTH']].sample(8, random_state=2), headers='keys', tablefmt='rst'))

... or from the `DIMENSION` column. This attribute is a string, from which we need to extract depth information. For this we use the following regular expression `str.extract('(\d+)')` that allow extracting the digits from the last 5 characters of the `DIMENSION` string text.

In [ ]:
idx2 = np.where((np.isnan(sondes['DEPTH'].values)))
sondes['DEPTH'].loc[idx2] = sondes['DIMENSION'].loc[idx2].str[-5:].str.extract('(\d+)', expand=False).astype(np.float32)
sondes.dropna(subset=['DEPTH'], inplace=True)
sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION', 'DEPTH']].sample(8, random_state=2)
# print(tabulate(sondes[['ALTITUDE_MIN','ALTITUDE_MAX','DIMENSION', 'DEPTH']].sample(8, random_state=2), headers='keys', tablefmt='rst'))

Now, we all the information needed to visualize teh geothermal probes dataset with `pydeck`. First we plot an histogram of the `DEPTH` attributes.

In [ ]:
bin = plt.hist(sondes['DEPTH'].values,30)
plt.xlabel('Depth [m]')

We have both positive values and negative value for `DEPTH`. This means that sometimes `ALTITUDE_MIN` and `ALTITUDE_MAX` have been inverted. We use the `abs` fonction to have positive values everywhere. 

In [ ]:
sondes['DEPTH'] = sondes['DEPTH'].abs()

bin = plt.hist(sondes['DEPTH'].values,30)
plt.xlabel('Depth [m]')


## Static visual exploration of the geothermal probes depth

We can use the `matplotlib` package to plot the coordinates of the probes cmapped bay the depth. The result will be something like this

In [ ]:
plt.scatter(sondes.lon, sondes.lat, c=sondes.DEPTH)
plt.colorbar()
plt.xlabel('Longitude')
plt.ylabel('Latitude')

This method gives us some information about the localization and depth, but it is visually difficult to interpret.

## Interactive visual exploration using `pydeck`

The "Layer" is a core concept of deck.gl. A deck.gl layer is a packaged visualization type that takes a collection of datums, associate each with positions, colors, extrusions, etc., and renders them on a map. For this visualization we use a `ColumnnLayer`. Read <a href="https://deck.gl/docs/api-reference/layers" target="_blank">this</a> for a deck.gl layer catalog overview.

In [ ]:
layer = pydeck.Layer(
    'ColumnLayer',  # `type` positional argument is here
    data=sondes,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    get_elevation='DEPTH',
    elevation_scale=25,
    radius=50,
    pickable=True,
    get_fill_color = [69,162,128,255],
    coverage=2)

view_state = pydeck.ViewState(
    longitude=6.183829,
    latitude=46.194656,
    zoom=10,
    min_zoom=5,
    max_zoom=15,
    pitch=25,
    bearing=0)

# Combined all of it and render a viewport
r = pydeck.Deck(layers=[layer],
                height=1500, 
                initial_view_state=view_state,
               )
r.to_html('geothermal-probes-viz.html')

with some customization ...

In [ ]:
COLOR_RANGE = [

    [49,54,149],
    [69,117,180],
    [116,173,209],
    [171,217,233],
    [224,243,248],
    [255,255,191],
    [254,224,144],
    [253,174,97],
    [244,109,67],
    [215,48,39],
    [165,0,38],
]


BREAKS = [30, 80, 130, 160,  190, 220, 250, 280,310,  340,370]


def color_scale(val):
    for i, b in enumerate(list(BREAKS)):
        if val < b:
            return COLOR_RANGE[i]
    return COLOR_RANGE[i]

sondes["fill_color"] = sondes["DEPTH"].apply(lambda x: color_scale(x))


In [ ]:
layer = pydeck.Layer(
    'ColumnLayer',  # `type` positional argument is here
    data=sondes,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    get_elevation='DEPTH',
    elevation_scale=25,
    radius=100,
    pickable=True,
    get_fill_color = "fill_color",

    coverage=2)


# Set the viewport location
view_state = pydeck.ViewState(
    longitude=6.183829,
    latitude=46.244656,
    zoom=10,
    
    min_zoom=0,
    max_zoom=24,
    pitch=45,
    bearing=0)

# Combined all of it and render a viewport
r = pydeck.Deck(layers=[layer],
                initial_view_state=view_state,
                map_style='light'
               )
r.to_html('geothermal-probes-viz-custom.html', open_browser=False, iframe_height=750)

In [ ]:
import pydeck as pdk

HEXAGON_LAYER_DATA = (
    "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv"  # noqa
)

# Define a layer to display on a map
layer = pdk.Layer(
    "HexagonLayer",
    data=sondes,
    get_position=['lon', 'lat'],
    auto_highlight=True,
    elevation_scale=10,
    pickable=True,
    elevation_range=[0, 1500],
    extruded=True,
    radius=500,
    coverage=1,
    color_range=COLOR_RANGE
)


# Render
r = pydeck.Deck(layers=[layer],
                initial_view_state=view_state,
#                 map_style='light'
                map_style='dark'
               )

# 
r.to_html("geothermal-probes-viz-hexagon.html.html",open_browser=False, iframe_height=750)

The resulting visualization is a `.html` file that can be opened in any browser (<a href="/static/images/visualization/geothermal-probes-viz-custom.html" target="_blank">click here to show the results in a browser</a>)


Data and code to reproduce this work can be found <a href="https://github.com/geoenergy/DST1_pydeck_visual/tree/master" target="_blank">here.</a>

A blog post about this tutorial can be found <a href="https://www.geomaap.io/blog/visualization/DST-viz-pydeck" target="_blank">here.</a>